In [9]:
# Import libraries
import pandas as pd
import numpy as np
from scipy.stats import shapiro 
import yfinance as yf
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from IPython.display import clear_output

In [10]:
def compute_returns(ticker, start_date, end_date):
  prices = pd.DataFrame(yf.download(ticker, start=start_date, end=end_date)['Close'])
  returns = prices.pct_change().dropna()
  returns.columns = ['Returns']

  # Calculate normal distribution with same mean and std as the returns
  returns['Normal Distribution'] = np.random.normal(loc=returns['Returns'].mean(), scale=returns['Returns'].std(), size=len(returns))

  # Line plot close prices
  fig = px.line(prices,
            x=prices.index,
            y=['Close'],
                  width=900)

  fig.update_layout(title="USD/BRL Close Price",
      xaxis_title="Date",
      yaxis_title="USD/BRL",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  # Line plot returns
  fig = px.line(returns,
            x=returns.index,
            y=['Returns'],
                  width=900)

  fig.update_layout(title="USD/BRL Daily Returns",
      xaxis_title="Date",
      yaxis_title="Returns",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  # Histogram of returns
  fig = px.histogram(returns,
            x=['Returns', 'Normal Distribution'],
                    barmode='overlay',
                  width=900)

  fig.update_layout(title="USD/BRL Returns Distribution",
      xaxis_title="Returns",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  return prices, returns

In [11]:
# Define ticker, import data and calculate returns
ticker = ['USDBRL=X']
prices, returns = compute_returns(ticker, start_date='2015-01-01', end_date='2023-10-31')

[*********************100%%**********************]  1 of 1 completed


In [12]:
# Perform Shapiro-Wilk test for normality
shapiro(returns)

ShapiroResult(statistic=0.9894906282424927, pvalue=5.2292286238167964e-18)

In [13]:
def compute_ewma_var(returns, lambda_value, time_horizon, risk_percentile):
  # Calculate squared returns
  returns['Squared Returns'] = returns['Returns']**2

  # Calculate EWMA volatility
  returns['EWMA Volatility'] = pd.Series(returns['Squared Returns']).ewm(alpha=1-lambda_value, adjust=False).mean().pow(0.5).shift(1)
  returns['Negative EWMA Volatility'] = -returns['EWMA Volatility']

  # Calculate the critical value for the standard normal distribution
  z_score_critical_value = norm.ppf(risk_percentile)

  # Calculate VaR
  returns['VaR'] = returns['EWMA Volatility'] * z_score_critical_value * np.sqrt(time_horizon)
  returns['VaR - Negative Returns'] = -returns['VaR']


  fig = px.line(returns,
            x=returns.index,
            y=['Returns', 'VaR', 'VaR - Negative Returns'],
                color_discrete_sequence=px.colors.qualitative.G10,
                  width=900)

  fig.update_layout(title="Predicted VaR x Actual Returns",
      xaxis_title="Date",
      yaxis_title="Returns",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  # Calculate percentage error
  returns['Actual > Predicted'] = returns['Returns'].abs() > returns['VaR']
  returns['Total Percentage Error'] = returns['Actual > Predicted'].expanding().sum()*100 / returns['Actual > Predicted'].expanding().count()
  returns['365D Rolling Percentage Error'] = returns['Actual > Predicted'].rolling('365D').sum()*100 / returns['Actual > Predicted'].rolling('365D').count()
  
  fig = px.line(returns,
           x=returns.index,
           y=['Total Percentage Error', '365D Rolling Percentage Error'],
                 width=900)

  fig.update_layout(title="VaR Estimation Error",
      xaxis_title="Date",
      yaxis_title="Percentage Error (%)",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  return returns


In [14]:
# Define smoothing parameter (lambda) and time horizon
lambda_value = 0.94  # You can adjust this value based on your preference
time_horizon = 1     # For daily VaR, set time_horizon to 1
risk_percentile = 0.95

ewma_var = compute_ewma_var(returns, lambda_value, time_horizon, risk_percentile)

In [15]:
df_ewma_var = ewma_var.copy()

In [16]:
def calculate_historical_var(df_returns, window, min_periods, percentile):
  df_returns['VaR'] = df_returns['Returns'].rolling(window=window, min_periods=min_periods).quantile(percentile).shift(1)
  df_returns['VaR - Negative Returns'] = df_returns['Returns'].rolling(window=window, min_periods=min_periods).quantile(1-percentile).shift(1)

  fig = px.line(df_returns,
            x=df_returns.index,
            y=['Returns', 'VaR', 'VaR - Negative Returns'],
                color_discrete_sequence=px.colors.qualitative.G10,
                  width=900)

  fig.update_layout(title="Predicted VaR x Actual Returns",
      xaxis_title="Date",
      yaxis_title="Returns",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  # Calculate percentage error
  df_returns['Actual > Predicted'] = (df_returns['Returns'] > df_returns['VaR']) | (df_returns['Returns'] < df_returns['VaR - Negative Returns'])
  df_returns['Total Percentage Error'] = df_returns['Actual > Predicted'].expanding().sum()*100 / df_returns['Actual > Predicted'].expanding().count()
  df_returns['365D Rolling Percentage Error'] = df_returns['Actual > Predicted'].rolling('365D').sum()*100 / df_returns['Actual > Predicted'].rolling('365D').count()

  fig = px.line(df_returns,
           x=df_returns.index,
           y=['Total Percentage Error', '365D Rolling Percentage Error'],
                 width=900)

  fig.update_layout(title="VaR Estimation Error",
      xaxis_title="Date",
      yaxis_title="Percentage Error (%)",
      legend=dict(title='Legend',
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1))

  fig.show()

  return df_returns

In [18]:
historical_var = calculate_historical_var(returns, window=1095, min_periods=90, percentile=0.95)

In [19]:
comparison = pd.merge(df_ewma_var, historical_var, left_index=True, right_index=True, suffixes=(' - EWMA', ' - Historical'))

fig = px.line(comparison,
           x=comparison.index,
           y=['Total Percentage Error - EWMA', '365D Rolling Percentage Error - EWMA', 'Total Percentage Error - Historical', '365D Rolling Percentage Error - Historical'],
                 width=900)

fig.update_layout(title="VaR Estimation Error",
    xaxis_title="Date",
    yaxis_title="Percentage Error (%)",
    legend=dict(title='',
              orientation="h",
              yanchor="bottom",
              y=1.02,
              xanchor="right",
              x=1))

fig.show()